<a href="https://colab.research.google.com/github/snehith26/natural-language-processing/blob/master/Text_Generation_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
try:
  %tensorflow_version 2.x
except:
  pass
import tensorflow as tf

In [0]:
import numpy as np
import os,time

In [0]:
#downloading the shakespeare dataset
filepath = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [40]:
text = ''.join(open(filepath, 'r', encoding='utf-8').readlines())
print(len(text))
# printing first 100 chars
print(text[:250])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [41]:
# get the vocab list
vocab = sorted(set(text))
print('Vocabulary length = {0}'.format(len(vocab)))

Vocabulary length = 65


In [0]:
# vectorize the text
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

# now that we have character to index and index to character conversion, vectorise the input text
text_as_int = np.array([char2idx[c] for c in text])


In [0]:
# the maximum length sentence we want for a single input in characters.

seq_length = 100
examples_per_epoch = len(text)//(seq_length + 1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)



In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [0]:
print(text[0:300])
for item in sequences.take(3):
  x = ''.join(idx2char[item.numpy()])
  print(x)
  print('--------')

In [0]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]

  return input_text, target_text

dataset = sequences.map(split_input_target)

# to see the dataset
# commenting for now
#list(dataset.as_numpy_iterator())[0]

In [80]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [81]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


In [86]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 64, 100), (64, 64, 100)), types: (tf.int64, tf.int64)>

In [0]:
# building the model
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)          
  ])